# Introduction

# EDA and Preprocessing

Use the SMILE Tiwtter dataset, avaiable at https://figshare.com/articles/smile_annotations_final_csv/3187909.

EDA is to familiarize oneself with: data structure/content and data distribution. It helps determine the scope of data we will focus on and also understand the limitation of analysis.

In [2]:
import torch
import pandas as pd
from tqdm.notebook import tqdm

In [4]:
df = pd.read_csv('smileannotationsfinal.csv',
                names=['id', 'text', 'category'])
df.set_index('id', inplace=True)

In [5]:
df.head()

,text,category
id,,
611857364396965889,@aandraous @britishmuseum @AndrewsAntonio Merc...,nocode
614484565059596288,Dorian Gray with Rainbow Scarf #LoveWins (from...,happy
614746522043973632,@SelectShowcase @Tate_StIves ... Replace with ...,happy
614877582664835073,@Sofabsports thank you for following me back. ...,happy
611932373039644672,@britishmuseum @TudorHistory What a beautiful ...,happy


In [6]:
# Remove multi-label records and nocode records
df = df[(~df['category'].str.contains('\|')) & (
    df['category'] != 'nocode')]

In [7]:
df['category'].value_counts()

happy           1137
not-relevant     214
angry             57
surprise          35
sad               32
disgust            6
Name: category, dtype: int64

In [14]:
possible_labels = df['category'].unique()

label_dict = {}
for index, label in enumerate(possible_labels):
    label_dict[label] = index

In [15]:
df['label'] = df['category'].replace(label_dict)

In [17]:
df['label'].value_counts()

0    1137
1     214
2      57
5      35
4      32
3       6
Name: label, dtype: int64

# Train/Validation Split

Split the sample to ensure that each class is properly distributed between training and validation sets.

There is no test set here (since there is no finetuning).

In [18]:
from sklearn.model_selection import train_test_split

In [23]:
X_train, X_val, y_train, y_val = train_test_split(
    df.index.values,
    df.label.values,
    test_size = 0.15,
    stratify = df.label.values
)

In [24]:
df['data_type'] = 'not_set'
df.loc[X_train, 'data_type'] = 'train'
df.loc[X_val, 'data_type'] = 'val'

In [25]:
df.groupby(['category', 'label', 'data_type']).count()

text
category     label data_type      
angry        2     train        48
                   val           9
disgust      3     train         5
                   val           1
happy        0     train       966
                   val         171
not-relevant 1     train       182
                   val          32
sad          4     train        27
                   val           5
surprise     5     train        30
                   val           5

# Load Tokenizer and Encode Data

Tokenizer is to convert free text into numbers. Encoder does a bit more than tokenizer to standardize the input data.

In [29]:
from transformers import BertTokenizer
from torch.utils.data import TensorDataset

In [30]:
tokenizer = BertTokenizer.from_pretrained(
    'bert-base-uncased',
    do_lower_case=True
)

I0816 23:15:47.993536 94808 filelock.py:274] Lock 1760844064976 acquired on C:\Users\johnn/.cache\torch\transformers\26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock
I0816 23:15:47.997526 94808 file_utils.py:748] https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt not found in cache or force_download set to True, downloading to C:\Users\johnn\.cache\torch\transformers\tmp2htwpcde


I0816 23:15:50.467163 94808 file_utils.py:752] storing https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt in cache at C:\Users\johnn/.cache\torch\transformers\26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
I0816 23:15:50.471153 94808 file_utils.py:755] creating metadata file for C:\Users\johnn/.cache\torch\transformers\26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084
I0816 23:15:50.476138 94808 filelock.py:318] Lock 1760844064976 released on C:\Users\johnn/.cache\torch\transformers\26bc1ad6c0ac742e9b52263248f6d0f00068293b33709fae12320c0e35ccfbbb.542ce4285a40d23a559526243235df47c5f75c197f04f37d1a0c124c32c9a084.lock
I0816 23:15:50.477136 94808 tokenization_utils_base.py:1254] loading file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-vocab.txt from cache at C:\Users\johnn/.c

In [32]:
encoded_data_train = tokenizer.batch_encode_plus(
    df[df['data_type'] == 'train']['text'].values,
    add_special_tokens=True, # tell ending/starting of sentence
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256, # No. of words
    return_tensors='pt'  # return pytorch type
)

encoded_data_val = tokenizer.batch_encode_plus(
    df[df['data_type'] == 'val']['text'].values,
    add_special_tokens=True, # tell ending/starting of sentence
    return_attention_mask=True,
    pad_to_max_length=True,
    max_length=256, # No. of words
    return_tensors='pt'  # return pytorch type
)

W0816 23:22:02.568247 94808 tokenization_utils_base.py:1447] Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.
W0816 23:22:03.200036 94808 tokenization_utils_base.py:1447] Truncation was not explicitely activated but `max_length` is provided a specific value, please use `truncation=True` to explicitely truncate examples to max length. Defaulting to 'longest_first' truncation strategy. If you encode pairs of sequences (GLUE-style) with the tokenizer you can select this strategy more precisely by providing a specific strategy to `truncation`.


In [33]:
# encoded_data is something like dictionary
input_ids_train = encoded_data_train['input_ids']
attention_masks_train = encoded_data_train['attention_mask']
labels_train = torch.tensor(df[df['data_type']=='train']['label'].values)

input_ids_val = encoded_data_val['input_ids']
attention_masks_val = encoded_data_val['attention_mask']
labels_val = torch.tensor(df[df['data_type']=='val']['label'].values)

In [34]:
# Encoded data
dataset_train = TensorDataset(input_ids_train, 
                             attention_masks_train,
                             labels_train)
dataset_val = TensorDataset(input_ids_val,
                            attention_masks_val,
                            labels_val)

# Set up BERT Pretrained Model

In [35]:
from transformers import BertForSequenceClassification

In [36]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(label_dict),
    output_attentions=False,
    output_hidden_states=False
)

I0817 11:24:24.971889 94808 filelock.py:274] Lock 1760840233144 acquired on C:\Users\johnn/.cache\torch\transformers\4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517.lock
I0817 11:24:24.973883 94808 file_utils.py:748] https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json not found in cache or force_download set to True, downloading to C:\Users\johnn\.cache\torch\transformers\tmpxc2fubnz


I0817 11:24:26.180711 94808 file_utils.py:752] storing https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json in cache at C:\Users\johnn/.cache\torch\transformers\4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0817 11:24:26.188692 94808 file_utils.py:755] creating metadata file for C:\Users\johnn/.cache\torch\transformers\4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0817 11:24:26.192678 94808 filelock.py:318] Lock 1760840233144 released on C:\Users\johnn/.cache\torch\transformers\4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517.lock
I0817 11:24:26.194672 94808 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at C:\U

I0817 11:24:26.475934 94808 filelock.py:274] Lock 1760844229656 acquired on C:\Users\johnn/.cache\torch\transformers\f2ee78bdd635b758cc0a12352586868bef80e47401abe4c4fcc3832421e7338b.36ca03ab34a1a5d5fa7bc3d03d55c4fa650fed07220e2eeebc06ce58d0e9a157.lock
I0817 11:24:26.477916 94808 file_utils.py:748] https://cdn.huggingface.co/bert-base-uncased-pytorch_model.bin not found in cache or force_download set to True, downloading to C:\Users\johnn\.cache\torch\transformers\tmplfaek122


I0817 11:31:08.770286 94808 file_utils.py:752] storing https://cdn.huggingface.co/bert-base-uncased-pytorch_model.bin in cache at C:\Users\johnn/.cache\torch\transformers\f2ee78bdd635b758cc0a12352586868bef80e47401abe4c4fcc3832421e7338b.36ca03ab34a1a5d5fa7bc3d03d55c4fa650fed07220e2eeebc06ce58d0e9a157
I0817 11:31:08.775274 94808 file_utils.py:755] creating metadata file for C:\Users\johnn/.cache\torch\transformers\f2ee78bdd635b758cc0a12352586868bef80e47401abe4c4fcc3832421e7338b.36ca03ab34a1a5d5fa7bc3d03d55c4fa650fed07220e2eeebc06ce58d0e9a157
I0817 11:31:08.779266 94808 filelock.py:318] Lock 1760844229656 released on C:\Users\johnn/.cache\torch\transformers\f2ee78bdd635b758cc0a12352586868bef80e47401abe4c4fcc3832421e7338b.36ca03ab34a1a5d5fa7bc3d03d55c4fa650fed07220e2eeebc06ce58d0e9a157.lock
I0817 11:31:08.781260 94808 modeling_utils.py:667] loading weights file https://cdn.huggingface.co/bert-base-uncased-pytorch_model.bin from cache at C:\Users\johnn/.cache\torch\transformers\f2ee78bdd635

W0817 11:31:11.457338 94808 modeling_utils.py:757] Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertForSequenceClassification: ['cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.seq_relationship.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.transform.LayerNorm.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPretraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
W0817 11:31:11.458334 94808 modeling_utils.p

# Create Data Loader

In [37]:
from torch.utils.data import DataLoader, RandomSampler, SequentialSampler

In [38]:
batch_size = 6
dataloader_train = DataLoader(
    dataset_train,
    sampler=RandomSampler(dataset_train),
    batch_size=batch_size
)

dataloader_val = DataLoader(
    dataset_val,
    sampler=RandomSampler(dataset_val),
    batch_size=32
)

# Set Up Optimizer and Scheduler

In [39]:
from transformers import AdamW, get_linear_schedule_with_warmup

In [40]:
optimizer = AdamW(
    model.parameters(),
    lr=1e-5,
    eps=1e-8 
)

In [41]:
epochs=10

scheduler = get_linear_schedule_with_warmup(
    optimizer,
    num_warmup_steps=0,
    num_training_steps=len(dataloader_train)*epochs
)

# Set Up Metrics

In [42]:
import numpy as np
from sklearn.metrics import f1_score

In [45]:
def f1_score_func(preds, labels):
    preds_flat = np.argmax(preds, axis=1).flatten() 
    # returns the indices of the maximum values along an axis
    # i.e. given there are multiple label categories to predict
    #      select the one with highest probability
    labels_flat = labels.flatten()
    return f1_score(labels_flat, preds_flat, average='weighted')

In [46]:
def accuracy_per_class(preds, labels):
    label_dict_inverse = {v: k for k, v in label_dict.items()}
    
    preds_flat = np.argmax(preds, axis=1).flatten()
    labels_flat = labels.flatten()
    
    for label in np.unique(labels_flat):
        y_preds = preds_flat[labels_flat == label]
        y_true = labels_flat[labels_flat == label]
        print(f'Class: {label_dict_inverse[label]}')
        print(f'Accuracy: {len(y_preds[y_preds==label])/len(y_preds)} \n')

# Create Training Loop

Approach adapted from an older version of HuggingFace `run_glue.py`

In [49]:
import random

seed_val=17
random.seed(seed_val)
np.random.seed(seed_val)
torch.manual_seed(seed_val)
torch.cuda.manual_seed_all(seed_val)

In [53]:
# device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
device = torch.device('cpu')
model.to(device)  # send the model to device

print(device)

cpu


In [61]:
def evaluate(dataload_val):
    
    model.eval()  
    # State the stage of model: eval not train, no update on weights
    #          and no backward propogation
    
    loss_val_total = 0
    predictions, true_vals = [], []
    
    for batch in tqdm(dataloader_val):
        
        batch = tuple(b.to(device) for b in batch)
        # Note: batch has 3 components defined before: input_id,
        #       attention_mask, data
        # Send each part to the device
        
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[1],
            'labels': batch[2]
        }
        
        with torch.no_grad():
            outputs = model(**inputs)  # get all elements in inputs

        loss = outputs[0]
        logits = outputs[1]
        loss_val_total += loss.item()
        
        logits = logits.detach().cpu().numpy()
        label_ids = inputs['labels'].cpu().numpy()
        predictions.append(logits)
        true_vals.append(label_ids)
        
    loss_val_avg = loss_val_total/len(dataloader_val)
    
    predictions = np.concatenate(predictions, axis=0)
    true_vals = np.concatenate(true_vals, axis=0)
    
    return loss_val_avg, predictions, true_vals

Training

In [68]:
for epoch in tqdm(range(1, epochs+1)):
    model.train()
    
    loss_train_total = 0
    
    progress_bar = tqdm(
        dataloader_train,
        desc='Epoch {:1d}'.format(epoch),
        leave=False, 
        # If true, keeps all traces of the progressbar
        # Otherwise, overwrite the progressbar
        disable=False
    )
    for batch in progress_bar:
        model.zero_grad()
        
        batch = tuple(b.to(device) for b in batch)
        
        inputs = {
            'input_ids': batch[0],
            'attention_mask': batch[0],
            'labels': batch[2]
        }
        
        outputs = model(**inputs)
        
        loss = outputs[0]
        loss_train_total += loss.item()
        loss.backward()
        
        torch.nn.utils.clip_grad_norm(model.parameters(), 1.0)  # clip gradients
        
        optimizer.step()
        scheduler.step()
        
        progress_bar.set_postfix({
            'training_loss': '{:.3f}'.format(loss.item()/len(batch))})
    
    torch.save(model.state_dict(), f'Models/BERT_ft_epoch{epoch}.model')
    
    tqdm.write(f'`nEpoch {epoch}')
    
    loss_train_avg = loss_train_total/len(dataloader_train)
    tqdm.write(f'Training loss: {loss_train_avg}')
    
    val_loss, predictions, true_vals = evaluate(dataloader_val)
    val_f1 = f1_score_func(predictions, true_vals)
    tqdm.write(f'Validation loss: {val_loss}')
    tqdm.write(f'F1 Score (weighted): {val_f1}')

C:\Anaconda\lib\site-packages\ipykernel_launcher.py:31: UserWarning: torch.nn.utils.clip_grad_norm is now deprecated in favor of torch.nn.utils.clip_grad_norm_.


`nEpoch 1
Training loss: 0.8178801484760784



Validation loss: 1.2261100837162562
F1 Score (weighted): 0.6656119824269878


C:\Anaconda\lib\site-packages\sklearn\metrics\classification.py:1437: UndefinedMetricWarning: F-score is ill-defined and being set to 0.0 in labels with no predicted samples.
  'precision', 'predicted', average, warn_for)


`nEpoch 2
Training loss: 0.8176338224893525


...


Validation loss: 1.226056524685451
F1 Score (weighted): 0.6656119824269878


`nEpoch 9
Training loss: 0.8144342758825847



Validation loss: 1.2267166376113892
F1 Score (weighted): 0.6656119824269878


`nEpoch 10
Training loss: 0.8159272000903175



Validation loss: 1.226496866771153
F1 Score (weighted): 0.6656119824269878



# Load and Evaluate the Model

In [62]:
model = BertForSequenceClassification.from_pretrained(
    'bert-base-uncased',
    num_labels=len(label_dict),
    output_attentions=False,
    output_hidden_states=False
)

model.to(device)

I0817 20:12:51.917353 94808 configuration_utils.py:264] loading configuration file https://s3.amazonaws.com/models.huggingface.co/bert/bert-base-uncased-config.json from cache at C:\Users\johnn/.cache\torch\transformers\4dad0251492946e18ac39290fcfe91b89d370fee250efe9521476438fe8ca185.7156163d5fdc189c3016baca0775ffce230789d7fa2a42ef516483e4ca884517
I0817 20:12:51.920345 94808 configuration_utils.py:300] Model config BertConfig {
  "architectures": [
    "BertForMaskedLM"
  ],
  "attention_probs_dropout_prob": 0.1,
  "gradient_checkpointing": false,
  "hidden_act": "gelu",
  "hidden_dropout_prob": 0.1,
  "hidden_size": 768,
  "id2label": {
    "0": "LABEL_0",
    "1": "LABEL_1",
    "2": "LABEL_2",
    "3": "LABEL_3",
    "4": "LABEL_4",
    "5": "LABEL_5"
  },
  "initializer_range": 0.02,
  "intermediate_size": 3072,
  "label2id": {
    "LABEL_0": 0,
    "LABEL_1": 1,
    "LABEL_2": 2,
    "LABEL_3": 3,
    "LABEL_4": 4,
    "LABEL_5": 5
  },
  "layer_norm_eps": 1e-12,
  "max_position_e

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [63]:
model

BertForSequenceClassification(
  (bert): BertModel(
    (embeddings): BertEmbeddings(
      (word_embeddings): Embedding(30522, 768, padding_idx=0)
      (position_embeddings): Embedding(512, 768)
      (token_type_embeddings): Embedding(2, 768)
      (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
      (dropout): Dropout(p=0.1, inplace=False)
    )
    (encoder): BertEncoder(
      (layer): ModuleList(
        (0): BertLayer(
          (attention): BertAttention(
            (self): BertSelfAttention(
              (query): Linear(in_features=768, out_features=768, bias=True)
              (key): Linear(in_features=768, out_features=768, bias=True)
              (value): Linear(in_features=768, out_features=768, bias=True)
              (dropout): Dropout(p=0.1, inplace=False)
            )
            (output): BertSelfOutput(
              (dense): Linear(in_features=768, out_features=768, bias=True)
              (LayerNorm): LayerNorm((768,), eps=1e-12, element

In [65]:
model.load_state_dict(
    torch.load('Models/BERT_ft_epoch1.model',
               map_location=torch.device('cpu'))
)

<All keys matched successfully>

In [66]:
_, predictions, true_vals = evaluate(dataloader_val)

In [67]:
accuracy_per_class(predictions, true_vals)

Class: happy
Accuracy: 1.0 

Class: not-relevant
Accuracy: 0.0 

Class: angry
Accuracy: 0.0 

Class: disgust
Accuracy: 0.0 

Class: sad
Accuracy: 0.0 

Class: surprise
Accuracy: 0.0 

